# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [26]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os
import openai
import json
from dotenv import load_dotenv

load_dotenv()
%env



{'DOCKER_BUILDKIT': '1',
 'ENABLE_DYNAMIC_INSTALL': 'true',
 'LESSOPEN': '| /usr/bin/lesspipe %s',
 'GITHUB_TOKEN': '<hidden>',
 'PYTHONIOENCODING': 'UTF-8',
 'GITHUB_CODESPACE_TOKEN': '<hidden>',
 'USER': 'codespace',
 'RVM_PATH': '/usr/local/rvm',
 'NVS_ROOT': '/usr/local/nvs',
 'HOSTNAME': 'codespaces-3f0539',
 'PIPX_HOME': '/usr/local/py-utils',
 'CONDA_SCRIPT': '/opt/conda/etc/profile.d/conda.sh',
 'SHLVL': '1',
 'GITHUB_CODESPACES_PORT_FORWARDING_DOMAIN': 'preview.app.github.dev',
 'HUGO_ROOT': '/home/codespace/.hugo',
 'HOME': '/home/codespace',
 'OLDPWD': '/',
 'ORYX_ENV_TYPE': 'vsonline-present',
 'NVM_BIN': '/usr/local/share/nvm/versions/node/v19.9.0/bin',
 'CODESPACES': 'true',
 'NVM_SYMLINK_CURRENT': 'true',
 'PIPX_BIN_DIR': '/usr/local/py-utils/bin',
 'DYNAMIC_INSTALL_ROOT_DIR': '/opt',
 'NVM_INC': '/usr/local/share/nvm/versions/node/v19.9.0/include/node',
 'rvm_stored_umask': '0022',
 'ORYX_DIR': '/usr/local/oryx',
 'GRADLE_HOME': '/usr/local/sdkman/candidates/gradle/curr

In [29]:
# Load config values

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv("OPENAI_API_BASE")
    
# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")



# Setting up the deployment name
chatgpt_model_name = os.getenv("CHATGPT_MODEL")

# Currently Chat Completions API have the following versions available: 2023-03-15-preview
openai.api_version = os.getenv("OPENAI_API_VERSION")


TypeError: 'function' object is not subscriptable

## 1.0 Create the system message for ChatGPT

In [5]:
base_system_message = "You are a helpful assistant.... Not"

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.... Not


## 2.0 Define helper functions



In [6]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [7]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=chatgpt_model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response['choices'][0]['message']['content']

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [8]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [9]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [10]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 38


In [11]:
max_response_tokens = 500

response = send_message(messages, chatgpt_model_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a helpful assistant.... Not

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great topic! Here are some points you could consider including in your post:

1. Introduction to AI and its growing importance in the workplace
2. Examples of AI technologies that are already impacting the workforce, such as chatbots, automation, and predictive analytics
3. Benefits of AI for businesses, such as increased efficiency and productivity
4. Concerns about the impact of AI on jobs and the workforce, including the potential for job displacement and the need for new skills and training
5. Ways in which businesses and individuals can prepare for the future of work in an AI-driven world, such as upskilling and retraining
6. Conclusion and thoughts on the future of work and AI

Good luck with your blog post!



## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [12]:
overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [13]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, chatgpt_model_name, max_response_tokens)

messages.append({"role": "assistant", "content": response})
print_conversation(messages)

Token count: 212
[SYSTEM]
You are a helpful assistant.... Not

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great topic! Here are some points you could consider including in your post:

1. Introduction to AI and its growing importance in the workplace
2. Examples of AI technologies that are already impacting the workforce, such as chatbots, automation, and predictive analytics
3. Benefits of AI for businesses, such as increased efficiency and productivity
4. Concerns about the impact of AI on jobs and the workforce, including the potential for job displacement and the need for new skills and training
5. Ways in which businesses and individuals can prepare for the future of work in an AI-driven world, such as upskilling and retraining
6. Conclusion and thoughts on the future of work and AI

Good luck with your blog post!

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
Gre

### Option 2: Keep the conversation within a given number of turns

In [15]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [16]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, chatgpt_model_name)
messages.append({"role": "assistant", "content": response})
print_conversation(messages)


[SYSTEM]
You are a helpful assistant.... Not

[USER]
I want to write a blog post about the impact of AI on the future of work.

[ASSISTANT]
Great topic! Here are some points you could consider including in your post:

1. Introduction to AI and its growing importance in the workplace
2. Examples of AI technologies that are already impacting the workforce, such as chatbots, automation, and predictive analytics
3. Benefits of AI for businesses, such as increased efficiency and productivity
4. Concerns about the impact of AI on jobs and the workforce, including the potential for job displacement and the need for new skills and training
5. Ways in which businesses and individuals can prepare for the future of work in an AI-driven world, such as upskilling and retraining
6. Conclusion and thoughts on the future of work and AI

Good luck with your blog post!

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
Great! In that case,